In [1]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.2 MB/s eta 0:00:00


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from torch.utils.data import DataLoader
from torchmetrics import Accuracy
from torchvision.datasets import ImageFolder
import pytorch_lightning as pl
import torch
from pathlib import Path
import math

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/celebrity-face-image-dataset/Celebrity Faces Dataset/Angelina Jolie/049_4d6df392.jpg
/kaggle/input/celebrity-face-image-dataset/Celebrity Faces Dataset/Angelina Jolie/051_268fdfd7.jpg
/kaggle/input/celebrity-face-image-dataset/Celebrity Faces Dataset/Angelina Jolie/045_c560251e.jpg
/kaggle/input/celebrity-face-image-dataset/Celebrity Faces Dataset/Angelina Jolie/052_6db5f5bf.jpg
/kaggle/input/celebrity-face-image-dataset/Celebrity Faces Dataset/Angelina Jolie/006_9135205d.jpg
/kaggle/input/celebrity-face-image-dataset/Celebrity Faces Dataset/Angelina Jolie/020_4c4b655f.jpg
/kaggle/input/celebrity-face-image-dataset/Celebrity Faces Dataset/Angelina Jolie/025_41cee764.jpg
/kaggle/input/celebrity-face-image-dataset/Celebrity Faces Dataset/Angelina Jolie/024_ca32be97.jpg
/kaggle/input/celebrity-face-image-dataset/Celebrity Faces Dataset/Angelina Jolie/092_26130bb1.jpg
/kaggle/input/celebrity-face-image-dataset/Celebrity Faces Dataset/Angelina Jolie/066_b378479b.jpg
/kaggle/in

In [3]:
device = torch.device("cpu")
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_built() and torch.backends.mps.is_available():
    device = torch.device("mps")
print(device)

cuda


In [4]:
path="/kaggle/input/celebrity-face-image-dataset/Celebrity Faces Dataset"
batch_size = 32
img_height = 224
img_width = 224

In [5]:
data_dir = Path(path)
ds = ImageFolder(data_dir)
indices = torch.randperm(len(ds)).tolist()
n_val = math.floor(len(indices) * .15)
print(n_val)
train_ds = torch.utils.data.Subset(ds, indices[:-n_val])
val_ds = torch.utils.data.Subset(ds, indices[-n_val:])

270


In [6]:
from datasets import load_dataset

ds_2 = load_dataset("imagefolder", data_dir=path)


Resolving data files:   0%|          | 0/1800 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [7]:
print(ds.classes)

['Angelina Jolie', 'Brad Pitt', 'Denzel Washington', 'Hugh Jackman', 'Jennifer Lawrence', 'Johnny Depp', 'Kate Winslet', 'Leonardo DiCaprio', 'Megan Fox', 'Natalie Portman', 'Nicole Kidman', 'Robert Downey Jr', 'Sandra Bullock', 'Scarlett Johansson', 'Tom Cruise', 'Tom Hanks', 'Will Smith']


In [10]:
ds_2['train'][0]['label']

0

In [33]:
label2id = {}
id2label = {}

for i, class_name in enumerate(ds.classes):
    label2id[class_name] = str(i)
    id2label[str(i)] = class_name

In [9]:
# from transformers import AutoImageProcessor, AutoModelForImageClassification, ViTFeatureExtractor,ViTForImageClassification

# image_processor = AutoImageProcessor.from_pretrained("jayanta/vit-base-patch16-224-in21k-face-recognition")
# model = AutoModelForImageClassification.from_pretrained("jayanta/vit-base-patch16-224-in21k-face-recognition",
#     num_labels=len(label2id),
#     label2id=label2id,
#     id2label=id2label
                                                       )

SyntaxError: incomplete input (563266819.py, line 7)

In [11]:
# from torchvision.transforms import RandomResizedCrop, ColorJitter, Compose

# size = (
#     image_processor.size["shortest_edge"]
#     if "shortest_edge" in image_processor.size
#     else (image_processor.size["height"], image_processor.size["width"])
# )

# _transforms = Compose([RandomResizedCrop(size), ColorJitter(brightness=0.5, hue=0.5)])

NameError: name 'image_processor' is not defined

In [ ]:
# def transforms(examples):
#     images = [_transforms(img.convert("RGB")) for img in examples["image"]]
#     examples["pixel_values"] = image_processor(images, do_resize=False, return_tensors="pt")["pixel_values"]
#     return examples

In [12]:
small_train_dataset = ds_2["train"].shuffle(seed=42)[:-n_val]
small_eval_dataset = ds_2["train"].shuffle(seed=42)[-n_val:]

In [13]:
# ds_2.set_transform(transforms)

NameError: name 'transforms' is not defined

In [14]:
# import matplotlib.pyplot as plt

# img = ds_2['train'][0]["pixel_values"]
# plt.imshow(img.permute(1, 2, 0))

KeyError: 'pixel_values'

In [15]:
print(type(ds))

<class 'torchvision.datasets.folder.ImageFolder'>


In [16]:
class ImageClassificationCollator:
    def __init__(self, feature_extractor):
        self.feature_extractor = feature_extractor

    def __call__(self, batch):
        encodings = self.feature_extractor([x[0] for x in batch], return_tensors='pt')
        encodings['labels'] = torch.tensor([x[1] for x in batch], dtype=torch.long)
        return encodings

In [17]:
print(type(train_ds))

<class 'torch.utils.data.dataset.Subset'>


In [18]:
# train_ds = tf.keras.utils.image_dataset_from_directory(
#   path,
#   validation_split=0.2,
#   subset="training",
#   seed=123,
#   image_size=(img_height, img_width),
#   batch_size=batch_size)

In [19]:
# val_ds = tf.keras.utils.image_dataset_from_directory(
#   path,
#   validation_split=0.2,
#   subset="validation",
#   seed=123,
#   image_size=(img_height, img_width),
#   batch_size=batch_size)

In [34]:
from transformers import AutoImageProcessor, AutoModelForImageClassification, ViTFeatureExtractor,ViTForImageClassification
feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224-in21k')
# # processor = AutoImageProcessor.from_pretrained("jayanta/vit-base-patch16-224-in21k-face-recognition")
model = ViTForImageClassification.from_pretrained("google/vit-base-patch16-224-in21k",
    num_labels=len(label2id),
    label2id=label2id,
    id2label=id2label)

config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [42]:
collator = ImageClassificationCollator(feature_extractor)
train_loader = DataLoader(train_ds, batch_size=8, collate_fn=collator, num_workers=2, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=8, collate_fn=collator, num_workers=2)

In [23]:
# inputs = image_processor(image, return_tensors="tf")

In [35]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [36]:
class Classifier(pl.LightningModule):

    def __init__(self, model, lr: float = 2e-5, **kwargs):
        super().__init__()
        self.save_hyperparameters('lr', *list(kwargs))
        self.model = model
        self.forward = self.model.forward
        self.val_acc = Accuracy(
            task='multiclass' if model.config.num_labels > 2 else 'binary',
            num_classes=model.config.num_labels
        )

    def training_step(self, batch, batch_idx):
        outputs = self(**batch)
        self.log(f"train_loss", outputs.loss)
        return outputs.loss

    def validation_step(self, batch, batch_idx):
        outputs = self(**batch)
        self.log(f"val_loss", outputs.loss)
        acc = self.val_acc(outputs.logits.argmax(1), batch['labels'])
        self.log(f"val_acc", acc, prog_bar=True)
        return outputs.loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.hparams.lr)

In [37]:
# from tensorflow.keras.optimizers import Adam
# # processed_ds=processor(train_ds)
# model.compile(optimizer=Adam(3e-5))
# model.fit(train_ds)
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", eval_strategy="epoch")

import evaluate

metric = evaluate.load("accuracy")

In [38]:
AUTOTUNE = tf.data.AUTOTUNE

# train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
# val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [39]:
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=small_train_dataset,
#     eval_dataset=small_eval_dataset,
#     compute_metrics=compute_metrics,
# )

In [40]:
# trainer.train()

In [43]:
pl.seed_everything(42)
classifier = Classifier(model, lr=2e-5)
trainer = pl.Trainer(accelerator='gpu', devices=1, precision=16, max_epochs=4)
trainer.fit(classifier, train_loader, val_loader)

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Training: |          | 0/? [00:00<?, ?it/s]

/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

In [46]:
val_batch = next(iter(val_loader))
count = 0
acc = 0
for ele in iter(val_loader):
    outputs = model(**ele)
    pred =outputs.logits.softmax(1).argmax(1)
    label = ele['labels']
    for pred,label in zip(pred,label):
        if pred == label:
            acc+=1
        count+=1

    print('Preds: ', outputs.logits.softmax(1).argmax(1))
    print('Labels:', ele['labels'])
print(f"accuracy: {(acc/count*100)}%")

Preds:  tensor([ 4,  5,  9,  9,  7,  2, 11, 16])
Labels: tensor([ 4,  5,  9,  9,  7,  2, 11,  2])
Preds:  tensor([ 5, 16,  5, 16, 13, 13, 14,  8])
Labels: tensor([ 5, 16,  5, 16, 13, 13, 14,  8])
Preds:  tensor([ 7, 15,  9,  1,  4, 14,  8,  5])
Labels: tensor([ 7, 15,  6,  1,  4, 14,  8,  5])
Preds:  tensor([12, 16, 14,  8,  7,  3,  7, 11])
Labels: tensor([12, 16, 14,  8,  7,  3,  3, 11])
Preds:  tensor([13, 11,  2,  1,  7, 12,  9, 16])
Labels: tensor([ 4, 11,  2,  1,  7, 12,  9, 16])
Preds:  tensor([13, 16, 14,  5,  4,  6,  5,  2])
Labels: tensor([13, 16, 14,  5,  4,  6,  5,  2])
Preds:  tensor([ 7,  0,  5, 13, 11,  9,  4,  9])
Labels: tensor([ 1,  8,  5, 13, 11,  9,  4,  9])
Preds:  tensor([12,  2, 13, 16, 11, 13, 15,  9])
Labels: tensor([12,  2, 13,  5, 11, 13, 15,  9])
Preds:  tensor([ 7,  1, 16, 13,  7, 16,  5, 11])
Labels: tensor([ 3,  3, 11, 13,  1,  2,  5, 11])
Preds:  tensor([ 7,  5,  1,  4, 16, 11, 14,  2])
Labels: tensor([ 7,  5,  1,  4, 16, 11, 14,  2])
Preds:  tensor([11, 